# Data Preprocessing for the Bachelorette Predictor
### Kwame V. Taylor

I need to preprocess the data from ```joined_df.csv``` to ready it for modeling. Then I will move on to continue my code in my main notebook, ```bachelorette-predictor.ipynb```, re-explore the data, and begin modeling.

### Set up Env

In [4]:
import numpy as np
import pandas as pd

### Import the tidied data

In [9]:
df = pd.read_csv('joined_df.csv', index_col=0)

In [10]:
df.head(30)

,Name,Age,Occupation,Hometown,ElimWeek,Season,ELIMINATION-1,ELIMINATION-2,ELIMINATION-3,ELIMINATION-4,...,DATES-1,DATES-2,DATES-3,DATES-4,DATES-5,DATES-6,DATES-7,DATES-8,DATES-9,DATES-10
CONTESTANT,,,,,,,,,,,,,,,,,,,,,
1_RYAN_S,Ryan Sutter,29,Firefighter,"Vail, Colorado",NaN,1,NaN,NaN,NaN,NaN,...,NaN,D14,D1,D1,D1,D1,NaN,NaN,NaN,NaN
1_CHARLIE_M,Charlie Maher,28,Financial Analyst,"Hermosa Beach, California",6.0,1,NaN,NaN,NaN,NaN,...,NaN,D14,D1,D1,D1,D1,NaN,NaN,NaN,NaN
1_RUSS_X,Russ,30,Writer,"San Rafael, California",5.0,1,NaN,NaN,NaN,NaN,...,NaN,D14,D1,D1,D1,NaN,NaN,NaN,NaN,NaN
1_GREG_T,Greg T.,28,Importer,"Manhattan, New York",4.0,1,NaN,NaN,NaN,E,...,NaN,D14,D5,D1,NaN,NaN,NaN,NaN,NaN,NaN
1_BOB_G,Bob Guiney,31,Mortgage Broker,"Ferndale, Michigan",3.0,1,NaN,NaN,E,NaN,...,NaN,D14,D5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1_JAMIE_B,Jamie Blyth,27,Professional Basketball Player,"Chicago, Illinois",3.0,1,NaN,NaN,E,NaN,...,NaN,D14,D5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1_MIKE_X,Mike,24,Sales and Consulting,"Cincinnati, Ohio",3.0,1,NaN,NaN,E,NaN,...,NaN,D14,D5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1_ROB_X,Rob,29,Computer Programmer,"Dallas, Texas",3.0,1,NaN,NaN,E,NaN,...,NaN,D14,D5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1_BRIAN_C,Brian C.,28,Mortgage Broker,"Dallas, Texas",2.0,1,NaN,E,NaN,NaN,...,NaN,D14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Summarize the data

In [15]:
df.Age.value_counts()

27    43
28    40
29    31
26    29
31    28
30    24
25    18
33    14
32    13
35     7
34     6
24     3
36     3
42     1
41     1
37     1
38     1
23     1
Name: Age, dtype: int64

### Get dummies

### Create features

### Create target feature (to predict on)

### Drop unnecessary features

### Scale the data

### Export data and make .py file